In [2]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from libraries.DataUtils import DataUtils
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from datetime import datetime

In [4]:
train = pd.read_csv("./data/input/train_cleaned.csv")
train = train.set_index('Id')
train.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
test = pd.read_csv("./data/input/test_cleaned.csv")
test = test.set_index('Id')
test.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,NoSewr,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,NoSewr,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [6]:
train.fillna("NA",inplace=True)
test.fillna("NA",inplace=True)
DataUtils.columns_nan_description(train)
DataUtils.columns_nan_description(test)

There are 80 columns and 0 (0.00%) of them have null values.
There are 79 columns and 0 (0.00%) of them have null values.


In [7]:
train_sale_price = train['SalePrice']
train = train.drop('SalePrice',axis=1)
train_test = pd.concat([train,test])
train_test.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NA,Reg,Lvl,AllPub,Inside,...,0,0,NA,NA,NA,0,2,2008,WD,Normal
2,20,RL,80.0,9600,Pave,NA,Reg,Lvl,AllPub,FR2,...,0,0,NA,NA,NA,0,5,2007,WD,Normal
3,60,RL,68.0,11250,Pave,NA,IR1,Lvl,AllPub,Inside,...,0,0,NA,NA,NA,0,9,2008,WD,Normal
4,70,RL,60.0,9550,Pave,NA,IR1,Lvl,AllPub,Corner,...,0,0,NA,NA,NA,0,2,2006,WD,Abnorml
5,60,RL,84.0,14260,Pave,NA,IR1,Lvl,AllPub,FR2,...,0,0,NA,NA,NA,0,12,2008,WD,Normal


In [8]:
categorical_columns = ['MSSubClass','MSZoning','Street','Alley','LotShape','LandContour','Utilities',
                        'LotConfig','LandSlope','Neighborhood','Condition1','Condition2','BldgType','HouseStyle',
                        'OverallQual','OverallCond','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType',
                        'ExterQual','ExterCond','Foundation','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1',
                        'BsmtFinType2','Heating','HeatingQC','CentralAir','Electrical','KitchenQual','Functional',
                        'FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','PavedDrive','PoolQC',
                        'Fence','MiscFeature','SaleType','SaleCondition']
numerical_columns = ['LotArea','YearBuilt','YearRemodAdd','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF',
                        'TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','BsmtFullBath','BsmtHalfBath',
                        'FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageYrBlt','GarageCars',
                        'GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea',
                        'MiscVal','MoSold','YrSold','LotFrontage']

In [9]:
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_X = pd.DataFrame(OH_encoder.fit_transform(train_test[categorical_columns]),columns = OH_encoder.get_feature_names(categorical_columns))
OH_cols_X.index = train_test.index
OH_cols_X.head(5)

,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
MinMax_Scaler = MinMaxScaler()
MM_cols_X = pd.DataFrame(MinMax_Scaler.fit_transform(train_test[numerical_columns]),columns = [numerical_columns])
MM_cols_X.index = train_test.index
MM_cols_X.head(5)

,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,LotFrontage
Id,,,,,,,,,,,,,,,,,,,,,
1,0.033420,0.949275,0.883333,0.12250,0.125089,0.0,0.064212,0.140098,0.109641,0.413559,...,0.000000,0.082210,0.000000,0.0,0.0,0.0,0.0,0.090909,0.50,0.150685
2,0.038795,0.753623,0.433333,0.00000,0.173281,0.0,0.121575,0.206547,0.194917,0.000000,...,0.209270,0.000000,0.000000,0.0,0.0,0.0,0.0,0.363636,0.25,0.202055
3,0.046507,0.934783,0.866667,0.10125,0.086109,0.0,0.185788,0.150573,0.123083,0.419370,...,0.000000,0.056604,0.000000,0.0,0.0,0.0,0.0,0.727273,0.50,0.160959
4,0.038561,0.311594,0.333333,0.00000,0.038271,0.0,0.231164,0.123732,0.131695,0.366102,...,0.000000,0.047170,0.268775,0.0,0.0,0.0,0.0,0.090909,0.00,0.133562
5,0.060576,0.927536,0.833333,0.21875,0.116052,0.0,0.209760,0.187398,0.170342,0.509927,...,0.134831,0.113208,0.000000,0.0,0.0,0.0,0.0,1.000000,0.50,0.215753


In [11]:
train_test_transformed = OH_cols_X.join(MM_cols_X)
train_test_transformed.head(5)

,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,...,"(WoodDeckSF,)","(OpenPorchSF,)","(EnclosedPorch,)","(3SsnPorch,)","(ScreenPorch,)","(PoolArea,)","(MiscVal,)","(MoSold,)","(YrSold,)","(LotFrontage,)"
Id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.000000,0.082210,0.000000,0.0,0.0,0.0,0.0,0.090909,0.50,0.150685
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.209270,0.000000,0.000000,0.0,0.0,0.0,0.0,0.363636,0.25,0.202055
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.000000,0.056604,0.000000,0.0,0.0,0.0,0.0,0.727273,0.50,0.160959
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.000000,0.047170,0.268775,0.0,0.0,0.0,0.0,0.090909,0.00,0.133562
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.134831,0.113208,0.000000,0.0,0.0,0.0,0.0,1.000000,0.50,0.215753


In [36]:
test_data_X = train_test_transformed.loc[test.index.tolist(),:]
train_data_X = train_test_transformed.loc[train.index.tolist(),:]

In [38]:
#Model 1
NN_model1 = tf.keras.models.Sequential()
NN_model1.add(tf.keras.layers.Dense(128,kernel_initializer = 'normal',input_dim = train_data_X.shape[1],activation='relu'))
NN_model1.add(tf.keras.layers.Dense(1,kernel_initializer = 'normal',activation='linear'))
NN_model1.compile(loss='mse', optimizer = tf.keras.optimizers.Adam(), metrics=[tf.keras.metrics.RootMeanSquaredError()])
NN_model1.fit(train_data_X, train_sale_price, epochs=500, batch_size=32, validation_split = 0.2)

ean_squared_error: 44663.6445 - val_loss: 2739856896.0000 - val_root_mean_squared_error: 52343.6406
Epoch 401/500
37/37 [==============================] - 0s 1ms/step - loss: 1989080832.0000 - root_mean_squared_error: 44599.1133 - val_loss: 2735090688.0000 - val_root_mean_squared_error: 52298.0938
Epoch 402/500
37/37 [==============================] - 0s 1ms/step - loss: 1983727872.0000 - root_mean_squared_error: 44539.0586 - val_loss: 2729981184.0000 - val_root_mean_squared_error: 52249.2227
Epoch 403/500
37/37 [==============================] - 0s 1ms/step - loss: 1978266112.0000 - root_mean_squared_error: 44477.7031 - val_loss: 2724667136.0000 - val_root_mean_squared_error: 52198.3438
Epoch 404/500
37/37 [==============================] - 0s 2ms/step - loss: 1972983680.0000 - root_mean_squared_error: 44418.2812 - val_loss: 2719623680.0000 - val_root_mean_squared_error: 52150.0117
Epoch 405/500
37/37 [==============================] - 0s 1ms/step - loss: 1967393024.0000 - root_mean_s

In [39]:
pred_test = NN_model1.predict(test_data_X)

sample_submission = pd.read_csv("./data/input/sample_submission.csv")
sample_submission['SalePrice'] = pred_test.squeeze()
sample_submission.to_csv(f"./data/output/submission_NN_model1_{datetime.now().strftime('%d_%m_%Y')}.csv",index=False)

In [42]:
#Model 2
NN_model2 = tf.keras.models.Sequential()
NN_model2.add(tf.keras.layers.Dense(128,kernel_initializer = 'normal',input_dim = train_data_X.shape[1],activation='relu'))
NN_model2.add(tf.keras.layers.Dense(256,kernel_initializer = 'normal',activation='relu'))
NN_model2.add(tf.keras.layers.Dense(256,kernel_initializer = 'normal',activation='relu'))
NN_model2.add(tf.keras.layers.Dense(1,kernel_initializer = 'normal',activation='linear'))
NN_model2.compile(loss='mse', optimizer = tf.keras.optimizers.Adam(), metrics=[tf.keras.metrics.RootMeanSquaredError()])
NN_model2.fit(train_data_X, train_sale_price, epochs=250, batch_size=32, validation_split = 0.2)

8
Epoch 150/250
37/37 [==============================] - 0s 3ms/step - loss: 311696672.0000 - root_mean_squared_error: 17654.9336 - val_loss: 1984304128.0000 - val_root_mean_squared_error: 44545.5273
Epoch 151/250
37/37 [==============================] - 0s 3ms/step - loss: 315433472.0000 - root_mean_squared_error: 17760.4473 - val_loss: 1959445376.0000 - val_root_mean_squared_error: 44265.6211
Epoch 152/250
37/37 [==============================] - 0s 3ms/step - loss: 312958976.0000 - root_mean_squared_error: 17690.6465 - val_loss: 1965659008.0000 - val_root_mean_squared_error: 44335.7539
Epoch 153/250
37/37 [==============================] - 0s 3ms/step - loss: 321196992.0000 - root_mean_squared_error: 17921.9688 - val_loss: 1991958016.0000 - val_root_mean_squared_error: 44631.3555
Epoch 154/250
37/37 [==============================] - 0s 3ms/step - loss: 307049472.0000 - root_mean_squared_error: 17522.8281 - val_loss: 1987966336.0000 - val_root_mean_squared_error: 44586.6172
Epoch 15

In [43]:
pred_test = NN_model2.predict(test_data_X)

sample_submission = pd.read_csv("./data/input/sample_submission.csv")
sample_submission['SalePrice'] = pred_test.squeeze()
sample_submission.to_csv(f"./data/output/submission_NN_model2_{datetime.now().strftime('%d_%m_%Y')}.csv",index=False)

In [51]:
#Model 3
NN_model3 = tf.keras.models.Sequential()
NN_model3.add(tf.keras.layers.Dense(train_data_X.shape[1],kernel_initializer = 'normal',input_dim = train_data_X.shape[1],activation='relu'))
NN_model3.add(tf.keras.layers.Dense(64,kernel_initializer = 'normal',activation='relu'))
NN_model3.add(tf.keras.layers.Dense(1,kernel_initializer = 'normal',activation='linear'))
NN_model3.compile(loss='mse', optimizer = tf.keras.optimizers.Adam(), metrics=[tf.keras.metrics.RootMeanSquaredError()])
NN_model3.fit(train_data_X, train_sale_price, epochs=250, batch_size=64, validation_split = 0.2)

7
Epoch 150/250
19/19 [==============================] - 0s 5ms/step - loss: 946998208.0000 - root_mean_squared_error: 30773.3359 - val_loss: 1779504640.0000 - val_root_mean_squared_error: 42184.1758
Epoch 151/250
19/19 [==============================] - 0s 4ms/step - loss: 941668928.0000 - root_mean_squared_error: 30686.6250 - val_loss: 1776117632.0000 - val_root_mean_squared_error: 42144.0117
Epoch 152/250
19/19 [==============================] - 0s 4ms/step - loss: 936890432.0000 - root_mean_squared_error: 30608.6660 - val_loss: 1772294400.0000 - val_root_mean_squared_error: 42098.6289
Epoch 153/250
19/19 [==============================] - 0s 3ms/step - loss: 933068032.0000 - root_mean_squared_error: 30546.1621 - val_loss: 1768414720.0000 - val_root_mean_squared_error: 42052.5234
Epoch 154/250
19/19 [==============================] - 0s 5ms/step - loss: 928391936.0000 - root_mean_squared_error: 30469.5254 - val_loss: 1764887680.0000 - val_root_mean_squared_error: 42010.5664
Epoch 15

In [52]:
pred_test = NN_model2.predict(test_data_X)

sample_submission = pd.read_csv("./data/input/sample_submission.csv")
sample_submission['SalePrice'] = pred_test.squeeze()
sample_submission.to_csv(f"./data/output/submission_NN_model3_{datetime.now().strftime('%d_%m_%Y')}.csv",index=False)